In [1]:
# import  os, glob
import  random, csv
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Activation, Dropout
from tensorflow.keras import layers, losses
from tensorflow import keras
import numpy as np
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping

import h5py
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub

In [2]:
import numpy as np

"""
Transformation from raw image data (nanomaggies) to the rgb values displayed
at the legacy viewer https://www.legacysurvey.org/viewer

Code copied from
https://github.com/legacysurvey/imagine/blob/master/map/views.py
"""

def sdss_rgb(imgs, bands, scales=None,
             m = 0.02):
    import numpy as np
    rgbscales = {'u': (2,1.5), #1.0,
                 'g': (2,2.5),
                 'r': (1,1.5),
                 'i': (0,1.0),
                 'z': (0,0.4), #0.3
                 }
    if scales is not None:
        rgbscales.update(scales)

    I = 0
    for img,band in zip(imgs, bands):
        plane,scale = rgbscales[band]
        img = np.maximum(0, img * scale + m)
        I = I + img
    I /= len(bands)
        
    # b,g,r = [rimg * rgbscales[b] for rimg,b in zip(imgs, bands)]
    # r = np.maximum(0, r + m)
    # g = np.maximum(0, g + m)
    # b = np.maximum(0, b + m)
    # I = (r+g+b)/3.
    Q = 20
    fI = np.arcsinh(Q * I) / np.sqrt(Q)
    I += (I == 0.) * 1e-6
    H,W = I.shape
    rgb = np.zeros((H,W,3), np.float32)
    for img,band in zip(imgs, bands):
        plane,scale = rgbscales[band]
        rgb[:,:,plane] = (img * scale + m) * fI / I

    # R = fI * r / I
    # G = fI * g / I
    # B = fI * b / I
    # # maxrgb = reduce(np.maximum, [R,G,B])
    # # J = (maxrgb > 1.)
    # # R[J] = R[J]/maxrgb[J]
    # # G[J] = G[J]/maxrgb[J]
    # # B[J] = B[J]/maxrgb[J]
    # rgb = np.dstack((R,G,B))
    rgb = np.clip(rgb, 0, 1)
    return rgb

def dr2_rgb(rimgs, bands, **ignored):
    return sdss_rgb(rimgs, bands, scales=dict(g=(2,6.0), r=(1,3.4), z=(0,2.2)), m=0.03)



In [3]:
file1=h5py.File(r"D:\dl\stardata\data\new_train_6w.h5",'r')
file2=h5py.File(r"D:\dl\stardata\data\new_val_1w.h5",'r')

In [4]:
sfr1=file1['specsfr_fib_p50']
lgm1=file1['lgm_fib_p50']
img1=file1['processed_images']

sfr2=file2['specsfr_fib_p50']
lgm2=file2['lgm_fib_p50']
img2=file2['processed_images']

In [5]:
import matplotlib.pyplot as plt
# 计算裁剪区域的左上角和右下角坐标
original_length=152
target_length=64
left = (original_length - target_length) // 2
top = (original_length - target_length) // 2
right = left + target_length
bottom = top + target_length

cropped_images = []
for image_array in img1:
    # 裁剪并添加到裁剪后的图像数组
    cropped_image = image_array[:, top:bottom, left:right]
    cropped_images.append(cropped_image)

img_rgb1=[dr2_rgb(i,['g','r','z'])  for i in cropped_images]
img_rgb1 = np.array(img_rgb1)

cropped_images = []
for image_array in img2:
    # 裁剪并添加到裁剪后的图像数组
    cropped_image = image_array[:, top:bottom, left:right]
    cropped_images.append(cropped_image)


img_rgb2=[dr2_rgb(i,['g','r','z'])  for i in cropped_images]
img_rgb2 = np.array(img_rgb2)

In [6]:
from sklearn.preprocessing import MinMaxScaler
#train sfr
scaler_sfr_train=MinMaxScaler()
scaler_sfr_1=np.array(sfr1).reshape(-1, 1)
scaler_sfr_train.fit(scaler_sfr_1)
scaler_sfr_1 = scaler_sfr_train.transform(scaler_sfr_1)
#train lgm
scaler_lgm_train=MinMaxScaler()
scaler_lgm_1=np.array(lgm1).reshape(-1, 1)
scaler_lgm_train.fit(scaler_lgm_1)
scaler_lgm_1 = scaler_lgm_train.transform(scaler_lgm_1)
# #val sfr
# scaler_sfr_val=MinMaxScaler()
scaler_sfr_2=np.array(sfr2).reshape(-1, 1)
scaler_sfr_2 = scaler_sfr_train.transform(scaler_sfr_2)
# #val lgm
# scaler_lgm_val=MinMaxScaler()
scaler_lgm_2=np.array(lgm2).reshape(-1, 1)
scaler_lgm_2 = scaler_lgm_train.transform(scaler_lgm_2)

In [7]:
# 32 16 8
from scipy.ndimage import zoom
img_zoom1=[zoom(i,(8/64,8/64,1)) for i in img_rgb1]
img_zoom2=[zoom(i,(8/64,8/64,1)) for i in img_rgb2]

img_zoom1=np.array(img_zoom1)
img_zoom2=np.array(img_zoom2)

In [8]:
with tf.device("CPU"):
    train_ds=tf.data.Dataset.from_tensor_slices((
        img_zoom1,
        {
            'output_1': scaler_lgm_1,
            'output_2': scaler_sfr_1
        }
    ))
    val_ds=tf.data.Dataset.from_tensor_slices((
        img_zoom2,
        {
            'output_1': scaler_lgm_2,
            'output_2': scaler_sfr_2
        }
    ))

In [9]:
seed_value = 42
tf.random.set_seed(seed_value)


batch_size = 256
train_ds = train_ds.shuffle(buffer_size=4000)  
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

val_ds = val_ds.batch(batch_size)
val_ds = val_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [12]:
#Learning Rate: 0.001, Dropout Rate: 0.25, Dense Units: 512
import tensorflow as tf

input_layer = tf.keras.layers.Input(shape=(8, 8, 3))

feature_extractor_layer = hub.KerasLayer("https://www.kaggle.com/models/google/efficientnet-v2/frameworks/TensorFlow2/variations/imagenet1k-b3-feature-vector/versions/2", trainable=True)(input_layer)

# Dropout层
dropout_layer = tf.keras.layers.Dropout(0.2864134988249096)(feature_extractor_layer)

# 全连接层
dense_layer = tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001))(dropout_layer)

# 输出层1，与标签1的类型匹配
output_1 = tf.keras.layers.Dense(1, activation='linear', name='output_1')(dense_layer)

# 输出层2，与标签2的类型匹配
output_2 = tf.keras.layers.Dense(1, activation='linear', name='output_2')(dense_layer)

# 创建多输出模型
final_model = tf.keras.models.Model(inputs=input_layer, outputs=[output_1, output_2])
#Learning Rate: 0.0003, Dropout Rate: 0.5, Dense Units: 128


In [13]:
#from tensorflow.keras.optimizers import legacy as keras_legacy_optimizers
#I 2023-10-23 21:50:25,794] Trial 6 finished with value: 0.447093928745786 and parameters: {'learning_rate': 3.759682546092012e-05, 'dropout_rate': 0.017501915825131043, 'rate': 0.004768418909358573, 'dense_units': 256}. Best is trial 6 with value: 0.447093928745786.
initial_learning_rate = 0.0003044714144499721
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#         initial_learning_rate, 
#         decay_steps=10,     
#         decay_rate=0.1,   
#         staircase=True)
boundaries=[30,45]
values=[initial_learning_rate,initial_learning_rate*0.1,initial_learning_rate*0.1*0.96]
learning_rate_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries, values)


optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)
#optimizer = keras_legacy_optimizers.Adam(learning_rate=1e-4)

final_model.compile(optimizer=optimizer,
               loss='mae',
               metrics=['mae'])

early_stopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0.005,
    patience=5
)
epoch=100
history=final_model.fit(train_ds,
            validation_data=val_ds,
            validation_freq=1,
            epochs=epoch,
            callbacks=[early_stopping])

Epoch 1/100
235/235 [==============================] - 34s 77ms/step - loss: 1.3907 - output_1_loss: 0.1547 - output_2_loss: 0.1810 - output_1_mae: 0.1547 - output_2_mae: 0.1810 - val_loss: 1.1766 - val_output_1_loss: 0.1351 - val_output_2_loss: 0.1713 - val_output_1_mae: 0.1351 - val_output_2_mae: 0.1713
Epoch 2/100
235/235 [==============================] - 16s 68ms/step - loss: 0.9067 - output_1_loss: 0.0851 - output_2_loss: 0.1164 - output_1_mae: 0.0851 - output_2_mae: 0.1164 - val_loss: 0.7903 - val_output_1_loss: 0.1080 - val_output_2_loss: 0.1301 - val_output_1_mae: 0.1080 - val_output_2_mae: 0.1301
Epoch 3/100
235/235 [==============================] - 16s 67ms/step - loss: 0.6163 - output_1_loss: 0.0753 - output_2_loss: 0.1069 - output_1_mae: 0.0753 - output_2_mae: 0.1069 - val_loss: 0.5553 - val_output_1_loss: 0.0991 - val_output_2_loss: 0.1262 - val_output_1_mae: 0.0991 - val_output_2_mae: 0.1262
Epoch 4/100
235/235 [==============================] - 16s 67ms/step - loss: 0.

In [14]:
from tensorflow.keras.models import load_model, save_model

final_model.save(r'model/8.h5')

In [15]:
file3=h5py.File(r'data/new_test_3w_old.h5','r')
img3=file3['processed_images']
sfr_true=np.array(file3['specsfr_fib_p50']).reshape(-1,1)
lgm_true=np.array(file3['lgm_fib_p50']).reshape(-1,1)
import matplotlib.pyplot as plt
# 计算裁剪区域的左上角和右下角坐标
original_length=152
target_length=64
left = (original_length - target_length) // 2
top = (original_length - target_length) // 2
right = left + target_length
bottom = top + target_length

cropped_images = []
for image_array in img3:
    # 裁剪并添加到裁剪后的图像数组
    cropped_image = image_array[:, top:bottom, left:right]
    cropped_images.append(cropped_image)


img_rgb3=[dr2_rgb(i,['g','r','z'])  for i in cropped_images]
img_rgb3=np.array(img_rgb3)

In [16]:
from scipy.ndimage import zoom
img_zoom3=[zoom(i,(8/64,8/64,1)) for i in img_rgb3]
img_zoom3=np.array(img_zoom3)

In [19]:
import tensorflow as tf
import tensorflow_hub as hub

# 自定义对象字典，用于告诉 TensorFlow 如何处理自定义层
custom_objects = {'KerasLayer': hub.KerasLayer}

# 指定保存的模型文件路径
model_path = '/Users/yuu/Downloads/matdisk/bound_8.h5'  # 替换为您的模型文件的实际路径

# 加载模型并传递自定义对象字典
final_model = tf.keras.models.load_model(model_path,custom_objects)


In [17]:
import pandas as pd

In [18]:
def count_mae(n1,n2):
    # 计算绝对差值
    n1=np.array(n1)
    n2=np.array(n2)
    absolute_errors = np.abs(n1 - n2)

    # 计算平均绝对误差（MAE）
    mae = np.mean(absolute_errors)

    print("MAE:", mae)

In [19]:
def predict_pix(model,img,name):
    y=final_model.predict(np.array(img))
    lgm_pred=np.array(y[0])
    lgm_pred=scaler_lgm_train.inverse_transform(lgm_pred)

    sfr_pred=np.array(y[1])
    sfr_pred=scaler_sfr_train.inverse_transform(sfr_pred)
    sd = np.std(np.array(lgm_true) - np.array(lgm_pred))
    print("MAE for lgm:")
    count_mae(lgm_true,lgm_pred)
    mse = np.mean((np.array(lgm_true) - np.array(lgm_pred)) ** 2)
    rmse=np.sqrt(mse)
    print(f'LGM的MSE是:{mse},RMSE是{rmse},SD是{sd}')
    sd = np.std(np.array(sfr_true) - np.array(sfr_pred))
    print("MAE for sfr:")
    count_mae(sfr_true,sfr_pred)

    mse = np.mean((np.array(sfr_true) - np.array(sfr_pred)) ** 2)
    rmse=np.sqrt(mse)
    print(f'SFR的MSE是:{mse},RMSE是{rmse},SD是{sd}')

    y_df=pd.DataFrame()
    y_df['lgm']=pd.DataFrame(lgm_pred)
    y_df['sfr']=pd.DataFrame(sfr_pred)

    y_df.to_csv(f'{name}_result.csv',index=False)